In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import scipy.stats 
from scipy.stats import bernoulli


import warnings
warnings.simplefilter('ignore')
binary = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'paid','activities', 'nursery', 'higher',
          'internet', 'romantic']
normal = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health']
nominal = ['Mjob', 'Fjob', 'reason', 'guardian']


studentData = pd.read_csv('/Users/kayhan/Downloads/student/student-mat.csv',delimiter = ';')
studentData = studentData.drop(columns = ['absences','G1','G2'])

less12 = studentData['G3'][studentData['G3'] <= 12].apply(lambda x : 0)
more12 = studentData['G3'][studentData['G3'] > 12].apply(lambda x : 1)
studentData['G3'].iloc[less12.index] = less12
studentData['G3'].iloc[more12.index] = more12

for i in binary:
    studentData[i] = pd.factorize(studentData[i])[0]
for i in nominal:
    studentData[i] = pd.factorize(studentData[i])[0]
    studentData[i] = studentData[i].apply(lambda x : x + 1)

    
len(studentData[studentData['G3'] == 1]), len(studentData[studentData['G3'] == 0]), len(studentData) , 131/395, 264/395

modData = studentData
cols = modData.columns

#CHANGE DATA SO THE NORMALS ARENT JUST 1-4 AND MAKE IT SO THAT ITS MORE LIKE A DIST
#THE REST IS FINE
#JUST CHANGE BINARY SO THAT ITS NOT GAY
def frequency():
    for i in cols:
        a = modData[i].value_counts()
        print(len(a))
        #print(a)
        #print('\n')

def norm(x,mean,std):
    exponent = np.exp(-((x-mean)**2 / (2 * std**2 )))
    return (1 / (np.sqrt(2 * np.pi) * std)) * exponent

def bern(x,p):
    return (p**x) * (1 - p)**(1-x)

def fit(data):
    classStats = {'binary': None ,'normal': None , 'nominal': None }
    
    pY = len(data[data['G3'] == 1]) / (len(data['G3']))
    pN = len(data[data['G3'] == 0]) / (len(data['G3']))
    
    tempDict = {}
    for temp in binary:
        probDict = {}
        #data[temp] = data[temp].apply(lambda x : 1)
        one = data[data['G3'] == 1]
        total = len(one)
        noOfSuc = len(one[one[temp] == 1])
        #probability of binary feature being 1 given that G3 is 1 
        #conditional probability of feature temp being 1 given that G3 is 1
        #and vice versa
        p1 = [noOfSuc / total, 1-noOfSuc / total]
        
        probDict[1] = p1
        #conditional probability of feature temp being 1 given that G3 is 0
        #and vice versa
        zero = data[data['G3'] == 0]
        total = len(zero)
        noOfSuc = len(zero[zero[temp] == 1])
        p0 = [noOfSuc / total, 1- noOfSuc / total] 
        probDict[0] = p0
        #first one in each one is p, second is 1-p
        tempDict[temp] = probDict
    classStats['binary'] = tempDict
    
    tempDict = {}
    for temp in normal:
        ones = data[data['G3'] == 1]
        oneMean = ones[temp].mean()
        oneSD = ones[temp].std()
#        USE LATER WHEN MAKING PREIDCTION THING 
#        print(temp,oneMean, oneSD)
#        ones = ones[temp].apply(lambda x : norm(x,oneMean, oneSD))
#        oneIndex = ones.index
#        data[temp].loc[oneIndex] = ones
        probDict = {}
        probDict[1]= [oneMean, oneSD]
#        classStats['normal'] = tempDict
        
        zeros = data[data['G3'] == 0]
        zeroMean = zeros[temp].mean()
        zeroSD = zeros[temp].std()
#         zeros = zeros[temp].apply(lambda x : norm(x,zeroMean, zeroSD))
#         zeroIndex = zeros.index
#         data[temp].loc[zeroIndex] = zeros
        probDict[0] = [zeroMean, zeroSD]
        tempDict[temp] = probDict
    classStats['normal'] = tempDict
        
    tempDict = {}
    for temp in nominal:
        totalLV = data[temp].value_counts()
        one = data[data['G3'] == 1][temp]
        oneLV = one.value_counts()
        zero = data[data['G3'] == 0][temp]
        zeroLV = zero.value_counts()

        probDict = {}
        levelDict = {}
        for j in range(1,len(totalLV) + 1):
            p1 = oneLV[j] / totalLV [j]
            
            levelDict[j] = p1
        probDict[1] = levelDict
        
        levelDict = {}
        for j in range(1,len(totalLV) + 1):
            p0 = zeroLV[j] / totalLV[j]
            levelDict[j] = p0
        probDict[0] = levelDict
        tempDict[temp] = probDict
    classStats['nominal'] = tempDict
#    print(nomStat)
    #print(classStats)
    prob = [pN,pY]
    return classStats, prob



def createFolds(data,p):
    x,y = data[cols[:-1]], data[cols[-1:]]
    return train_test_split(x,y, test_size = p)
stats,prob = fit(studentData)
#takes in fitted modle aka the stats 
def predict(model, data,y,prob):
    y = y['G3']
    one = data.copy()
    zero = data.copy()
    for temp in binary:
        #finding probs for 1!!!!!
        p = model['binary'][temp][1][0]
        one[temp] = one[temp].apply(lambda x : bern(x,p))
        #samething for 0
        p = model['binary'][temp][0][0]
        zero[temp] = zero[temp].apply(lambda x : bern(x,p))
    for temp in nominal:
        p = model['nominal'][temp][1]
        one[temp] = one[temp].apply(lambda x : p[x])
        
        p = model['nominal'][temp][0]
        
        zero[temp] = zero[temp].apply(lambda x : p[x])
    
    for temp in normal:
        p = model['normal'][temp][1]
        mean = p[0] 
        std = p[1]
        one[temp] = one[temp].apply(lambda x : norm(x,mean,std))
        p = model['normal'][temp][0]
        mean = p[0] 
        std = p[1]
        zero[temp] = zero[temp].apply(lambda x : norm(x, mean, std))
    
    one = one.apply(np.log)
    oneSum = one.sum(axis = 1)
    pY = prob[1]
    pN = prob[0]
    oneSum = oneSum.apply(lambda x : x + pY)

    zero = zero.apply(np.log)
    zeroSum = zero.sum(axis = 1)
    zeroSum = zeroSum.apply(lambda x :x + pN)
    guess = []
    for jennie, soyeon in zip(oneSum,zeroSum):
        if jennie > soyeon:
            guess.append(1)
        else:
            guess.append(0)
    guess = np.array(guess)
    score = 0
    for peach,daisy in zip(guess,y):
        if peach == daisy:
            score += 1
    score = score / len(y)
    return guess,score


cross = []

for i in range(10):
    x_train, x_test, y_train, y_test = createFolds(studentData, 0.15)
    x_train['G3'] = y_train
    classStats,prob = fit(x_train)
    x_train = x_train.drop(columns = ['G3'])
    t,score = predict(classStats, x_train,y_train,prob)
    cross.append(score)
print('Training Score: ' + str(np.mean(cross)))
print(np.std(cross))

cross = []
for i in range(10):
    x_train, x_test, y_train, y_test = createFolds(studentData, 0.15)
    x_train['G3'] = y_train
    classStats,prob = fit(x_train)
    x_train = x_train.drop(columns = ['G3'])
    c,score = predict(classStats, x_test, y_test, prob)
    cross.append(score)
print('Test Score: ' + str(np.mean(cross)))
print(np.std(cross))

Training Score: 0.7471641791044775
0.008234695059184328
Test Score: 0.6866666666666668
0.05567764362830023


In [7]:
from sklearn.naive_bayes import GaussianNB
cross = []
for i in range(10):
    x_train, x_test, y_train, y_test = createFolds(studentData, 0.15)
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    score = gnb.score(x_train,y_train)
    cross.append(score)
print('SK Learn Training Score: ' + str(np.mean(cross)))
print(np.std(cross))

cross = []
for i in range(10):
    x_train, x_test, y_train, y_test = createFolds(studentData, 0.15)
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    score = gnb.score(x_test,y_test)
    cross.append(score)
print('SK Learn Test Score: ' + str(np.mean(cross)))
print(np.std(cross))

SK Learn Training Score: 0.6074626865671642
0.09221175278294619
SK Learn Test Score: 0.635
0.04561310727801334


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import scipy.stats 
from scipy.stats import bernoulli


import warnings
warnings.simplefilter('ignore')
binary = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'paid','activities', 'nursery', 'higher',
          'internet', 'romantic']
normal = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health']
nominal = ['Mjob', 'Fjob', 'reason', 'guardian']
nominal += normal
normal = []

studentData = pd.read_csv('/Users/kayhan/Downloads/student/student-mat.csv',delimiter = ';')
studentData = studentData.drop(columns = ['absences','G1','G2'])

less12 = studentData['G3'][studentData['G3'] <= 12].apply(lambda x : 0)
more12 = studentData['G3'][studentData['G3'] > 12].apply(lambda x : 1)
studentData['G3'].iloc[less12.index] = less12
studentData['G3'].iloc[more12.index] = more12

for i in binary:
    studentData[i] = pd.factorize(studentData[i])[0]
for i in nominal:
    studentData[i] = pd.factorize(studentData[i])[0]
    studentData[i] = studentData[i].apply(lambda x : x + 1)

    
len(studentData[studentData['G3'] == 1]), len(studentData[studentData['G3'] == 0]), len(studentData) , 131/395, 264/395

modData = studentData
cols = modData.columns

#CHANGE DATA SO THE NORMALS ARENT JUST 1-4 AND MAKE IT SO THAT ITS MORE LIKE A DIST
#THE REST IS FINE
#JUST CHANGE BINARY SO THAT ITS NOT GAY
def frequency():
    for i in cols:
        a = modData[i].value_counts()
        print(len(a))
        #print(a)
        #print('\n')

def norm(x,mean,std):
    exponent = np.exp(-((x-mean)**2 / (2 * std**2 )))
    return (1 / (np.sqrt(2 * np.pi) * std)) * exponent

def bern(x,p):
    return (p**x) * (1 - p)**(1-x)

def fit(data):
    classStats = {'binary': None ,'normal': None , 'nominal': None }
    
    pY = len(data[data['G3'] == 1]) / (len(data['G3']))
    pN = len(data[data['G3'] == 0]) / (len(data['G3']))
    
    tempDict = {}
    for temp in binary:
        probDict = {}
        #data[temp] = data[temp].apply(lambda x : 1)
        one = data[data['G3'] == 1]
        total = len(one)
        noOfSuc = len(one[one[temp] == 1])
        #probability of binary feature being 1 given that G3 is 1 
        #conditional probability of feature temp being 1 given that G3 is 1
        #and vice versa
        p1 = [noOfSuc / total, 1-noOfSuc / total]
        
        probDict[1] = p1
        #conditional probability of feature temp being 1 given that G3 is 0
        #and vice versa
        zero = data[data['G3'] == 0]
        total = len(zero)
        noOfSuc = len(zero[zero[temp] == 1])
        p0 = [noOfSuc / total, 1- noOfSuc / total] 
        probDict[0] = p0
        #first one in each one is p, second is 1-p
        tempDict[temp] = probDict
    classStats['binary'] = tempDict
        
    tempDict = {}
    for temp in nominal:
#         totalLV = data[temp].value_counts()
#         one = data[data['G3'] == 1][temp]
#         oneLV = one.value_counts()
#         keys = np.unique(data[temp])
#         zero = data[data['G3'] == 0][temp]
#         zeroLV = zero.value_counts()
#         probDict = {}
#         levelDict = {}
        
#         for j in keys:
#             try:
#                 p1 = oneLV[j] / totalLV [j]
#                 levelDict[j] = p1
#             except KeyError:
#                 levelDict[j] = 0
#         probDict[1] = levelDict
#         levelDict = {}
#         for j in keys:
#             try:
#                 p0 = zeroLV[j] / totalLV[j]
#                 levelDict[j] = p0        
#             except:
#                 levelDict[j] = 0
#         probDict[0] = levelDict
        
        levels = np.unique(data[temp])
        one = data[data['G3'] == 1][temp].value_counts()
        zero = data[data['G3'] == 0][temp].value_counts()
        oneDict = {}
        zeroDict = {}
        for key in levels:
            if key not in one:
                one[key] = 0
            if key not in zero:
                zero[key] = 0
            p1 = one[key] / sum(one)
            p0 = zero[key] / sum(zero)
            oneDict[key] = p1
            zeroDict[key] = p0
        probDict = {1: oneDict, 0:zeroDict}
        tempDict[temp] = probDict
    classStats['nominal'] = tempDict
#    print(nomStat)
    #print(classStats)
    prob = [pN,pY]
    return classStats, prob



def createFolds(data,p):
    x,y = data[cols[:-1]], data[cols[-1:]]
    return train_test_split(x,y, test_size = p)
stats,prob = fit(studentData)
#print(stats['nominal']['age'])
#takes in fitted modle aka the stats 
def predict(model, data,y,prob):
    y = y['G3']
    one = data.copy()
    zero = data.copy()
    for temp in binary:
        #finding probs for 1!!!!!
        p = model['binary'][temp][1][0]
        one[temp] = one[temp].apply(lambda x : bern(x,p))
        #samething for 0
        p = model['binary'][temp][0][0]
        zero[temp] = zero[temp].apply(lambda x : bern(x,p))

    
    for temp in nominal:
        p = model['nominal'][temp][1]
        
        def probP(p,x):
            if x not in p:
                return 0
            else:
                return p[x]
            
        one[temp] = one[temp].apply(lambda x : probP(p,x))    
        p = model['nominal'][temp][0]
        zero[temp] = zero[temp].apply(lambda x : probP(p,x))    
    

    one = one.apply(np.log)
    oneSum = one.sum(axis = 1)
    pY = prob[1]
    pN = prob[0]
    oneSum = oneSum.apply(lambda x : x + pY)

    zero = zero.apply(np.log)
    zeroSum = zero.sum(axis = 1)
    zeroSum = zeroSum.apply(lambda x :x + pN)
    guess = []
    for jennie, soyeon in zip(oneSum,zeroSum):
        if jennie > soyeon:
            guess.append(1)
        else:
            guess.append(0)
    guess = np.array(guess)
    score = 0
    for peach,daisy in zip(guess,y):
        if peach == daisy:
            score += 1
    score = score / len(y)
    return guess,score


cross = []

for i in range(10):
    x_train, x_test, y_train, y_test = createFolds(studentData, 0.15)
    x_train['G3'] = y_train
    classStats,prob = fit(x_train)
    x_train = x_train.drop(columns = ['G3'])
    t,score = predict(classStats, x_train,y_train,prob)
    cross.append(score)
print('Training Score: ' + str(np.mean(cross)))
print(np.std(cross))

cross = []
for i in range(10):
    x_train, x_test, y_train, y_test = createFolds(studentData, 0.15)
    x_train['G3'] = y_train
    classStats,prob = fit(x_train)
    x_train = x_train.drop(columns = ['G3'])
    c,score = predict(classStats, x_test, y_test, prob)
    cross.append(score)
print('Test Score: ' + str(np.mean(cross)))
print(np.std(cross))

Training Score: 0.7298507462686566
0.011657089068517403
Test Score: 0.6683333333333333
0.06121455890735651
